In this notebook I follow the algorithms and some of the prose in the article "Tangent-space methods for uniform matrix product states" by Laurens Vanderstraeten, Jutho Haegeman and Frank Verstraete published in January 2019 at
SciPost: https://doi.org/10.21468/SciPostPhysLectNotes.7. (arXiV version: https://arxiv.org/abs/1810.07006.)

This article has algorithms for both uniform-gauge and mixed-gauge TDVP.
The mixed-gauge version is inverse free, at the cost of having a step in the time evolution that, at the time of my writing this notebook, remains elusive (magic) to me.

I have so far only implemented "one site" TDVP, where the bond dimension remains constant.
The same article has an algorithm (#3) for maximizing the overlap between two MPS, so in principle, one could implement that and have varying bond dimensions.
*(gokhan, 24.05.2021: I don't understand anymore "maximizing the overlap between two MPS => variable bond dimension".)*
Note that a subset of the authors has recently (January 2020) put a paper to arXiv titled "Tangent-space methods for truncating uniform MPS", https://arxiv.org/abs/2001.11882.
For anyone who wants to vary the bond dimension it might be worth a read beforehand.

The packages KrylovKit and TensorOperations are also by Jutho Haegeman.
For time evolution, I use "exponentiate" from KrylovKit to
exponentiate the effective Hamiltonians, one of which is a 3-tensor, in
a matrix-free manner.
I use it also for the matrix-free computations of fixed points of
transfer tensors and similar constructs.

----
# Basic machinery
We will often need to vectorize 3-tensors and take QR, RQ, polar and singular value decompositions.
I will define the functions for them.
First importing libraries:

In [1]:
# Packages used in this code
using LinearAlgebra # Used for svd, qr.
using TensorOperations # Used for @tensor
using KrylovKit # Used for eigsolve, linsolve and exponentiate
using EllipsisNotation # Used for ... slicing of arrays

## Site tensors
We start out with a uniform gauge where the site tensor is $A$.
In the code, site tensors, by my convention, have memory layout A[i,j,k] where $i$ is the left leg, $j$ is the physical leg, and $k$ is the right leg.

Let's set the problem dimensions.

In [2]:
dH = 2; # Local Hilbert space dimension
dBond = 3; # Bond dimension

Let's create $A$ randomly, with contents drawn from $[0,1) + i[0,1)$.
The contents will change upon normalization (and gauge changes).

*TODO: Input/output such initial conditions. See for example Python codes at my finite TDVP repository.*

In [3]:
A = rand(ComplexF64, (dBond, dH, dBond));

## Vectorizing site tensors
We will often need to pack the left leg and physical leg or the physical leg and the right leg to one "packed leg" in order to perform matrix decompositions, and then unpack them.
Let's define some functions for that.

In [4]:
function leftPack(A)
    dBond, dH = size(A)[1:2];
    Amatrix = reshape(A, (dBond*dH, dBond));
    return Amatrix;
end;

function leftUnpack(Amatrix)
    dC, dBond = size(Amatrix);
    dH = dC ÷ dBond;
    A = reshape(Amatrix, (dBond, dH, dBond));
    return A;
end;

function rightPack(A)
    dBond, dH = size(A)[1:2];
    Amatrix = reshape(A, (dBond, dBond*dH));
end;

function rightUnpack(Amatrix)
    dBond, dC = size(Amatrix);
    dH = dC ÷ dBond;
    A = reshape(Amatrix, (dBond, dH, dBond));
end;

## Gauge-free QR
**Nothing** works if we don't remove the gauge freedom in QR.
We can't even switch to left/right-orthonormal forms from a uniform form as iterative methods don't converge
due to sign flips.
In the functions below I make sure that the diagonal elements in R are are non-negative.

In [12]:
function qrGauged(a; tol=eps(Float64))
    f = qr(a)
    q = Matrix(f.Q);
    r = f.R;
    D = size(r)[1];
    rDiag = zeros((D,D));
    for i=1:D
        check = r[i, i];
        # If an item on the diagonal is negative,
        # record that to flip signs later
        if abs(check) > tol && real(check) < 0
            rDiag[i, i] = -1;
        else
            rDiag[i,i] = 1;
        end;
    end;
    # Flip signs now
    r = rDiag * r;
    q = q * rDiag;

    return q, r;
end;

function rqGauged(a; tol=eps(Float64))
    
    # RQ from QR
    # https://leohart.wordpress.com/2010/07/23/rq-decomposition-from-qr-decomposition/
    reversed_a = reverse(a, dims=1);

    f = qr(transpose(reversed_a));
    q = Matrix(f.Q);
    r = f.R;

    r = reverse(transpose(r), dims=1);
    r = reverse(r, dims=2);
    q = reverse(transpose(q), dims=1);
    
    D = size(r)[2];
    rDiag = zeros((D,D));
    for i=1:D
        check = r[i, i];
        if abs(check) > tol && real(check) < 0
            rDiag[i, i] = -1;
        else
            rDiag[i,i] = 1;
        end;
    end;
    r = r * rDiag;
    q = rDiag * q;
    return r, q;
end;

## Gauge-free SVD
Julia (or Python) doesn't have an SVD-independent implementation of the polar decomposition which is needed for inverse-free TDVP.
That it is dependent on SVD is potentially numerically unstable.
Note that SVD is an intermediate step (in easy implementations) for the polar decomposition that is needed later, and is also necessary for two site TDVP to truncate the bond dimension (which this report doesn't do).

In [13]:
function svdGauged(a; tol=eps(Float64))
    F = svd(a);
    u = F.U;
    s = F.S;
    vh = F.Vt;
    # Form singular values as matrix
    sMatrix = zeros((size(u)[2], size(vh)[1]));
    for i=1:length(s)
        sMatrix[i, i] = s[i];
    end;
    D = min(size(a)[1], size(a)[2]);
    # Scan through left singular vectors
    # Make first non-zero component real and positive
    for i=1:D
        ui = u[:, i];
        mags = abs.(ui);
        # Find the first component with magnitude larger than tol
        idx = findfirst(mags .> tol);
        val = ui[idx];
        # Find its phase
        angle = atan(imag(val), real(val));
        # If the phase is not zero
        if abs(angle) > tol
            phase = exp(1im * angle);
            # Multiply the columns of u with the opposite phase
            u[:, i] *= conj(phase);
            # Multiply the rows of vh with the phase
            vh[i, :] *= phase;
        end;

        # Drop the imaginary part of the first non-zero element of u completely
        u[idx, i] = real(u[idx, i]);
    end;

    return u, sMatrix, vh;
end;

## Gauge-free polar decomposition
This will be used to find $A_L, A_R$, given $A_C, C$.
It's implemented through SVD.

In [7]:
function polarGauged(a; side="right")
    m, n = size(a);
    k = min(m, n);
    us, ss, vhs = svdGauged(a);
    u = us[:, 1:k] * vhs[1:k, :];
    if side == "right"
        p = vhs'[:, 1:k] * ss[1:k, 1:k] * vhs[1:k, :];
    elseif side == "left"
        p = us[:, 1:k] * ss[1:k, 1:k] * us'[1:k, :];
    end;

    return u, p;
end;

# Finding canonical forms
We're now ready to switch from a uniform gauge to the left-canonical or right-canonical form, or to their mixture, the mixed gauge.
The algorithms for these don't need to be optimized as they are done once for the initial condition.
They involve iterative methods in order to have nice convergences, avoiding square roots and inverses.

## Left-canonical form *(Algorithm 1)*
Given $A$, we want left-unitary $A_L$ and a matrix $L$ such that
$$A_L L = L A$$
holds.
We solve this problem iteratively where each iteration has two main steps:
 * $A_L^{i+1}, \tilde{L}^{i+1} = QR(L^i A)$
 * $L^{i+1} = $ fixedPoint($A_L^{{i+1}^\dagger} \tilde{L}^{i+1} A$)

In [8]:
function leftOrthonormalize(A; L0=nothing, tol=eps(Float64), resDiv=10)
    dBond, dH = size(A)[1:2];
    # If L0 is provided, take that as the initial guess
    # Otherwise create a random matrix as L0
    if L0 == nothing
        # Get a random matrix
        L = rand(ComplexF64, (dBond, dBond));
        L /= norm(L);
    else
        # Take the provided guess but normalize it
        L = copy(L0) / norm(L0);
    end;
    Lold = copy(L);
    # First guess for A_L and L
    @tensor LtimesA[i,j,k] := L[i,l] * A[l,j,k];
    ALmatrix, L = qrGauged(leftPack(LtimesA))
    AL = leftUnpack(ALmatrix);
    normL = norm(L);
    L /= normL;
    res = maximum(abs.(L-Lold));
    while res > tol
        # Transfer map for the left fixed point
        function leftFixedPointMap(Lguess)
            @tensor contraction[k,l] := Lguess[i,j]*conj(AL)[i,m,k]*A[j,m,l];
            return contraction;
        end;
        # Find its dominant eigenvector
        # Note that the eigensolver tolerance increases gradually
        L = eigsolve(leftFixedPointMap, L, tol=res/resDiv)[2][..,1];
        # Refine with QR
        L = qrGauged(L)[2];
        L /= norm(L);
        Lold = copy(L)
        # Iterate with QR
        @tensor LtimesA[i,j,k] = L[i,l] * A[l,j,k];
        ALmatrix, L = qrGauged(leftPack(LtimesA))
        AL = leftUnpack(ALmatrix);
        normL = norm(L);
        L /= normL;
        res = maximum(abs.(L-Lold));
    end;
    
    # A / normL would be a normalized site tensor
    return AL, L, normL;
end;

## Right-canonical form *(Algorithm 1)*
Given $A$, we want right-unitary $A_R$ and a matrix $R$ such that
$$R A_R = A R$$
holds.
We solve this problem similar to the left-canonical form.
There are two main steps:
 * $\tilde{R}^{i+1}, A_R^{i+1} = RQ(A R^i)$
 * $R^{i+1} = $ fixedPoint($A \tilde{R}^{i+1} A_R^{{i+1}^\dagger}$)

In [9]:
function rightOrthonormalize(A; R0=nothing, tol=eps(Float64), resDiv=10)
    dBond, dH = size(A)[1:2];
    # If R0 is provided, take that as the initial guess
    # Otherwise create a random matrix as R0
    if R0 == nothing
        # Get a random matrix
        R = rand(ComplexF64, (dBond, dBond));
        R /= norm(R);
    else
        # Take the provided guess but normalize it
        R = copy(R0) / norm(R0);
    end;
    Rold = copy(R);
    # First guess for A_R and R
    @tensor AtimesR[i,j,k] := A[i,j,l] * R[l,k];
    R, ARmatrix = rqGauged(rightPack(AtimesR))
    AR = rightUnpack(ARmatrix);
    normR = norm(R);
    R /= normR;
    res = maximum(abs.(R-Rold));
    while res > tol
        # Transfer map for the right fixed point
        function rightFixedPointMap(Rguess)
            @tensor contraction[i,j] := conj(AR)[j,m,l] * A[i,m,k] * Rguess[k,l];
            return contraction;
        end;
        # Find its dominant eigenvector
        # Note that the eigensolver tolerance increases gradually
        R = eigsolve(rightFixedPointMap, R, tol=res/resDiv)[2][..,1];
        # Refine with RQ
        R = rqGauged(R)[1];
        R /= norm(R);
        Rold = copy(R)
        # Iterate with RQ
        @tensor AtimesR[i,j,k] = A[i,j,l] * R[l,k];
        R, ARmatrix = rqGauged(rightPack(AtimesR))
        AR = rightUnpack(ARmatrix);
        normR = norm(R);
        R /= normR;
        res = maximum(abs.(R-Rold));
    end;
    
    # A / normR would be a normalized site tensor
    return AR, R, normR;
end;

## Mixed form *(Algorithm 2)*
We want to use $A_L$ and $A_R$ together. For that, we need a center-site tensor $A_C$, and the matrix $C$ which does
$$ A_L C = A_C = C A_R.$$

In [10]:
function mixedCanonical(A, tol=eps(Float64), resDiv=10)
    dBond, dH = size(A)[1:2];
    # Compute left and right orthonormal forms
    AL, _, normA = leftOrthonormalize(A, tol=tol, resDiv=resDiv);
    AR, C, _ = rightOrthonormalize(AL, tol=tol, resDiv=resDiv);
    # Diagonalize C
    u, C, vh = svdGauged(C);
    # Absorb u and vh to AL and AR
    @tensor AL[i,j,k] = u'[i,l]*AL[l,j,m]*u[m,k];
    @tensor AR[i,j,k] = vh[i,l]*AR[l,j,m]*vh'[m,k];
    # Compute AC
    @tensor AC[i,j,k] := AL[i,j,l]*C[l,k];

    return AL, AC, AR, C, normA;
end;

### Sanity checks for canonical forms

In [15]:
AL_, L = leftOrthonormalize(A)[1:2];
AR_, R = rightOrthonormalize(A)[1:2];
AL, AC, AR, C, normA = mixedCanonical(A);
println("The numbers below should be on the order of machine epsilon.")
@tensor test[k,l] := conj(AL)[i,m,k] * AL[i,m,l];
println("Left orthonormality: ", maximum(abs.(test-I)));
@tensor test[i,j,k] := L[i,l] * A[l,j,k] / normA - AL_[i,j,l] * L[l,k];
println("Uniform to left: ", maximum(abs.(test)));
@tensor test[i,j] := conj(AR)[j,m,k] * AR[i,m,k];
println("Right orthonormality: ", maximum(abs.(test-I)));
@tensor test[i,j,k] := R[i,l] * AR_[l,j,k] - A[i,j,l] * R[l,k] / normA;
println("Uniform to right: ", maximum(abs.(test)));
@tensor test[i,j,k] := AL[i,j,l]*C[l,k] - C[i,l] * AR[l,j,k];
println("Left/right to center: ", maximum(abs.(test)));

The numbers below should be on the order of machine epsilon.
Left orthonormality: 4.652682298944613e-16
Uniform to left: 1.8253379971363983e-16
Right orthonormality: 8.881784197001252e-16
Uniform to right: 2.1179449947025655e-16
Left/right to center: 7.850462293418876e-16


# Finding $A_L, A_R$ given $A_C, C$ *(Algorithm 5)*
The "inverse-free" part of the algorithm is here.
Time evolution is done for $A_C$ and $C$: From them, $A_L$ and $A_R$ found as the solutions that minimize

$$||A_C - A_L C||_2,$$
$$||A_C - C A_R||_2.$$

These are solved simultaneously from the polar decompositions of $A_C$ and $C$, (part of) the proof is *Theorem IX.7.2* in *Matrix Analysis, Bhatia, 1952, Springer*, so-called a way of finding "the best unitary approximation to a matrix."
Alex and I have spent many hours trying to prove this result using this theorem, in the end we did, but I am not sure if I could prove it again
quickly if I wanted to.
I omit that proof here.

In [16]:
function minACC(AC, C)
    # See "Variational optimization algorithms for uniform matrix product states"
    # by Zauner-Stauber et al.
    # for an explanation.
    # arXiv: https://arxiv.org/abs/1701.07035
    # PhysRevB: https://doi.org/10.1103/PhysRevB.97.045145

    dBond, dH = size(AC)[1:2];
    ULAC = leftUnpack(polarGauged(leftPack(AC), side="right")[1])
    ULC = polarGauged(C, side="right")[1]
    @tensor AL[i,j,k] := ULAC[i,j,l] * ULC'[l,k];
    URAC = rightUnpack(polarGauged(rightPack(AC), side="left")[1])
    URC = polarGauged(C, side="left")[1]
    @tensor AR[i,j,k] := URC'[i,l] * URAC[l,j,k];

    return AL, AR;
end;

Let's see if we get the same $A_L$ and $A_R$ from before when we use it:

In [17]:
AL_, AR_ = minACC(AC, C);
println("The numbers below should be on the order of machine epsilon.")
println("AL: ", maximum(abs.(AL_-AL)));
println("AR: ", maximum(abs.(AR_-AR)));

The numbers below should be on the order of machine epsilon.
AL: 4.503336797797561e-16
AR: 5.176433455214083e-15


# Effective Hamiltonian(s)
This is the most elaborate part of the implementation. What follows are series of functions that, when stiched together, form the action of the Hamiltonian *and then* the MPS projector applied to a state.
The end result is a matrix-free form of the effective Hamiltonian(s), which can then be exponentiated.
There are in fact two such operators, one for $A_C$ ($G_1$) and one for $C$ ($G_2$).

Note that the article has a rather serious "accessibility" problem:
The first time they define these effective Hamiltonians (eqs. 170 and 172), they have $A_C$ and $C$ already acted on.
That is, they are not Hamiltonians, they can't be exponentiated and then made to hit on $A_C$ and $C$.
However, later they then have a change of mind and redefine them without $A_C$ and $C$ (without telling how to do it) in eqs. 174 and 175.
Then they can be exponentiated.

The functions below construct $G_1$ and $G_2$ as operators that are to act on $A_C$ and $C$.

These involve iterative methods, most importantly in ```getLh``` and ```getRh``` to get around having to take inverses, and also in ```ELLp``` and ```ERRp``` to find the fixed points of the transfer tensors.
One needs initial guesses for iterative methods, so for the case when there is none, these functions take the (pseudo) inverses (for ```getLh``` and ```getRh```) or use random guesses (for ```ELLp``` and ```ERRp```).
For the subsequent timesteps the result of the last timestep can be given back as guesses, and they tend to converge in a few steps.
This is why all the functions below have optional arguments with default values of "nothing".

There is room for optimization in these, particularly where ```@tensor``` is called with ```:=```, as that reallocates memory.

In [ ]:
function ELLp(AL; r0=nothing, tol=eps(Float64))
    """Computes (1 - T_L') where T_L' is the transfer tensor in the left
    canonical form with the dominant component subtracted out.
    
    Part of the rhs of eq. (169)-1.
    """

    dBond, dH = size(AL)[1:2];
    # Compute the right fixed point of AL's transfer tensor
    # Transfer tensor:
    @tensor TL[i,j,k,l] :=  conj(AL)[j,m,l] * AL[i,m,k];
    # Its action on the right fixed point
    function rightFixedPointMap(rGuess)
        @tensor contraction[i,j] := TL[i,j,k,l]*rGuess[k,l];
        return contraction;
    end;

    if r0 == nothing
        # Random Hermitian matrix
        r = Matrix(Hermitian(rand(ComplexF64, (dBond, dBond))));
    else
        r = copy(r0);
    end;
    # Find the fixed point
    r = eigsolve(rightFixedPointMap, r, tol=tol)[2][..,1];
    # Normalize it with its trace
    r /= tr(r);
    # Make it exactly Hermitian
    r = Matrix(Hermitian(r));

    # Negate and remove the fixed point projector
    @tensor domP[i,j,k,l] := r[i,j] * Matrix{ComplexF64}(I, dBond, dBond)[k,l];
    TLPP = -TL + domP;
    # Add identity
    @tensor idP[i,j,k,l] := Matrix{ComplexF64}(I, dBond, dBond)[i,j] * Matrix{ComplexF64}(I, dBond, dBond)[k,l];
    TLP = idP + TLPP;

    return TLP, r;
end;

In [ ]:
function ERRp(AR; l0=nothing, tol=eps(Float64))
    """Computes (1 - T_R') where T_R' is the transfer tensor in the right
    canonical form with the dominant component subtracted out.
    
    Part of the rhs of eq. (169)-2.
    """

    dBond, dH = size(AR)[1:2];
    # Compute the left fixed point of AR's transfer tensor
    @tensor TR[i,j,k,l] := conj(AR)[i,m,k] * AR[j,m,l];
    # Its action on the left fixed point
    function leftFixedPointMap(lGuess)
        @tensor contraction[k,l] := lGuess[i,j]*TR[i,j,k,l];
        return contraction;
    end;

    if l0 == nothing
        # Random Hermitian matrix
        l = Matrix(Hermitian(rand(ComplexF64, (dBond, dBond))));
    else
        l = copy(l0);
    end;
    
    # Find the fixed point
    l = eigsolve(leftFixedPointMap, l, tol=tol)[2][..,1];
    # Normalize it with its trace
    l /= tr(l);
    # Make it exactly Hermitian
    l = Matrix(Hermitian(l));

    # Negate and remove the fixed point projector
    @tensor domP[i,j,k,l] := Matrix{ComplexF64}(I, dBond, dBond)[i,j] * l[k,l];
    TRPP = -TR + domP;
    # Add identity
    @tensor idP[i,j,k,l] := Matrix{ComplexF64}(I, dBond, dBond)[i,j] * Matrix{ComplexF64}(I, dBond, dBond)[k,l];
    TRP = idP + TRPP;

    return TRP, l;
end;

In [ ]:
function getLh(AL, h; r0=nothing, Lh0=nothing, tol=eps(Float64))
    """Computes Lh in eq. (169), a contribution to the tangent space projector.
        h is the local two site Hamiltonian.
    """

    dBond, dH = size(AL)[1:2];
    TLP, r = ELLp(AL, r0=r0);
    # Contraction around the local Hamiltonian
    @tensor hc[i,j] := conj(AL)[k,l,q] * conj(AL)[q,m,j] * h[l,m,n,o] * AL[k,n,p] * AL[p,o,i];

    # Equation to be solved
    function LhEq(Lhguess)
        @tensor preres[k,l] := Lhguess[j,i] * TLP[i,j,k,l];
        return preres - hc;
    end;

    if Lh0 == nothing
        # Compute the inverse
        Lh = Matrix(transpose(reshape(transpose(reshape(hc, (dBond^2)))*pinv(reshape(TLP, (dBond^2, dBond^2))),(dBond, dBond))));
    else
        Lh = copy(Lh0);
    end;

    Lh = linsolve(LhEq, zeros(ComplexF64, size(Lh)), Lh, tol=tol)[1];
    sol = Matrix(Hermitian(reshape(Lh, (dBond, dBond))));

    return sol, r;
end;

In [ ]:
function getRh(AR, h; l0=nothing, Rh0=nothing, tol=eps(Float64))
    """Computes Rh in eq. (169), a contribution to the tangent space projector.
        h is the local two site Hamiltonian.
    """

    dBond, dH = size(AR)[1:2];
    TRP, l = ERRp(AR, l0=l0);
    # Contraction around the local Hamiltonian
    @tensor hc[k,l] := conj(AR)[k,i,p] * conj(AR)[p,j,q] * h[i,j,m,n] * AR[l,m,o] * AR[o,n,q];

    # Equation to be solved
    function RhEq(Rhguess)
        @tensor preres[i,j] := TRP[i,j,k,l] * Rhguess[l,k];
        return preres - hc;
    end;

    if Rh0 == nothing
        # Compute the inverse
        Rh = Matrix(transpose(reshape(pinv(reshape(TRP, (dBond^2, dBond^2)))*reshape(hc, (dBond^2)),(dBond, dBond))));
    else
        Rh = copy(Rh0);
    end;
    
    Rh = linsolve(RhEq, zeros(ComplexF64, size(Rh)), Rh, tol=tol)[1];
    sol = Matrix(Hermitian(Rh));

    return sol, l;
end;

In [ ]:
function getG1L(AL, h)
    """Computes acting-from-left part of G_1 in eq. (170) (second term).
        So this would hit on A_C to the right.
        Part of the tangent space projector.
    """

    dBond, dH = size(AL)[1:2];
    @tensor G1L[i,j,k,l] := conj(AL)[m,n,i]*h[n,j,p,l]*AL[m,p,k];

    # This is supposed to be Hermitian.
    G1Lmatrix = Matrix(Hermitian(reshape(G1L, (dH * dBond, dH * dBond))));
    G1L = reshape(G1Lmatrix, (dBond, dH, dBond, dH));

    return G1L;
end;

In [ ]:
function getG1R(AR, h)
    """Computes acting-from-right part of G_1 in eq. (170) (first term).
        So this would hit on A_C to the left.
        Part of the tangent space projector.
    """

    dBond, dH = size(AR)[1:2];
    @tensor G1R[i,j,k,l] := conj(AR)[l,o,m] * h[k,o,i,p] * AR[j,p,m];

    # This is supposed to be Hermitian.
    G1Rmatrix = Matrix(Hermitian(reshape(G1R, (dH * dBond, dH * dBond))));
    G1R = reshape(G1Rmatrix,(dH,dBond,dH,dBond));

    return G1R;
end;

The function below is for the time evolution fo $C$.
It is related to $G_2$ in eq. 172, but what is in the article is really not convincing when one looks at the tangent space projector in eq. 91.
Why would one contract with $A_L$ (eq. 172) specifically?
Why not $A_R$?
Looking at eq. 91, I constructed $G_2$ myself analogously to eq. 170.
Its last two terms are both there, but only one term in place of the first two terms is there, where $C$ goes in between $A_L$ and $A_R$.
The function below defines this latter operator.

In [ ]:
function getG2M(AL, AR, h)
    """Computes part of G_2 eq. (172).
        This would take C in the "middle".
        This doesn't exist in the article directly, to understand,
        see the tangent space projector in eq. 91.
    """

    dBond, dH = size(AL)[1:2];
    @tensor G2M[i,j,k,l] := conj(AL)[m,n,i] * conj(AR)[j,o,r] * h[n,o,p,q] * AL[m,p,k] * AR[l,q,r];

    # This is supposed to be Hermitian.
    G2Mmatrix = Matrix(Hermitian(reshape(G2M, (dBond^2, dBond^2))));
    G2M = reshape(G2Mmatrix,(dBond, dBond, dBond, dBond));
    
    return G2M;
end;

# Hamiltonian
Before we move on with the time evolution, we need a Hamiltonian.
Here, I define a general nearest-neighbour type Hamiltonian for two-state systems, without quantum number conservation.
Generalizing to next-neighbour Hamiltonians may require a change in the definition of the effective Hamiltonian.
Larger local Hilbert spaces with nearest-neighbour interactions, again without quantum number conservation, should still work.

In [ ]:
# Pauli operators
Sx = 0.5*[0 1; 1 0];
Sy = 0.5*1im * [0 -1; 1 0];
Sz = 0.5*[1 0; 0 -1];
identity = Matrix{ComplexF64}(I, 2, 2);

In [ ]:
function hamiltonian(;jx=0, jy=0, jz=1.0, hx=1.0, hy=0, hz=0.4)
    """Hamiltonian for
        j_i S_i S_{i+1} + h_i S_i
        type interactions.

        ham[ijkl] = <ij|H|kl>.
    """

    ham = zeros(Complex{Float64},(2, 2, 2, 2));
    function addTerm(op1, op2)
        @tensor res[i,j,k,l] := op1[i,k]*op2[j,l];
        return res;
    end;
        
    ham += jx * addTerm(Sx,Sx);
    ham += jy * addTerm(Sy,Sy);
    ham += jz * addTerm(Sz,Sz);
    ham += hx * addTerm(Sx,identity);
    ham += hy * addTerm(Sy,identity);
    ham += hz * addTerm(Sz,identity);

    return ham;
end;

# Time stepper
We have everything we need for time evolution.
The equations of motion for $A_C$ and $C$ are

$$\dot{A_C} = -i G_1[A_L,A_R] A_C,$$
$$\dot{C} = +i G_2[A_L, A_R] C.$$

Note the sign difference.
I made explicit the $A_L$ and $A_R$ dependence of the operators in brackets.
Now comes the "magic".
The basic time step of $\delta t$ is made with
 * $A_C(t+\delta t) = e^{-i G_1[A_L(t), A_R(t)]\, \delta t} A_C(t)$
 * $C(t+\delta t) = e^{-i G_2[A_L(t), A_R(t)]\, \delta t} C(t)$
 * $A_L(t+\delta t), A_R(t+\delta t) = \text{minACC}(AC(t+\delta t, C(t+\delta t))$

Note that the exponents have the same sign, unlike before.

In [ ]:
function eulerStepExp(AL, AC, AR, C, h; tstep=0.01, l0=nothing, r0=nothing, Lh0=nothing, Rh0=nothing, tol=eps(Float64))
    """Time evolve AC and C, use them to find evolved AL, AR.
    """
    
    # The four parts of G_1
    G1L = getG1L(AL, h);
    G1R = getG1R(AR, h);
    Lh, r = getLh(AL, h, r0=r0, Lh0=Lh0);
    Rh, l = getRh(AR, h, l0=l0, Rh0=Rh0);

    function hamAC(AC_)
        res = zeros(ComplexF64,(dBond,dH,dBond));
        @tensor res[i,j,m] += G1L[i,j,k,l] * AC_[k,l,m];
        @tensor res[i,l,m] += AC_[i,j,k] * G1R[j,k,l,m];
        @tensor res[i,k,l] += Lh[i,j]  * AC_[j,k,l];
        @tensor res[i,j,l] += AC_[i,j,k] * Rh[k,l];
        
        return res;
    end;

    # Forward AC
    AC_ = exponentiate(hamAC,-1im * tstep, AC,ishermitian=true,tol=tol / abs(tstep))[1];
    
    # The three parts of G_2
    G2M = getG2M(AL, AR, h);
    function hamC(C_)
        res = zeros(ComplexF64,(dBond,dBond));
        @tensor res[i,j] += G2M[i,j,k,l] * C_[k,l];
        @tensor res[i,k] += Lh[i,j] * C_[j,k];
        @tensor res[i,k] += C_[i,j] * Rh[j,k];

        return res;
        
    end;
    
    # Backward C
    C_ = exponentiate(hamC,-1im * tstep,C,ishermitian=true,tol=tol / abs(tstep))[1];

    # Find evolved A_L and A_R using evolved A_C and C
    AL_, AR_ = minACC(AC_, C_);

    return AL_, AC_, AR_, C_, l, r, Lh, Rh;
end;

# Testing the time stepper

Since we have a finite-chain TDVP code, we can test against it.
The observables should agree, and further, for my dynamical systems related demands, everything should evolve continously.

*To be continued. The Python implementation had passed the tests.*

In [ ]:
h = hamiltonian();

In [ ]:
AL_, AC_, AR_, C_, l, r, Lh, Rh = eulerStepExp(AL, AC, AR, C, h, tstep=0.01);